In [1]:
from data_utils import prepare_partitioned_dataset

In [1]:
from model import *

/home/namvq/.cache/pypoetry/virtualenvs/niid-bench-orV17zke-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
net = ResNet18(2)

/home/namvq/.cache/pypoetry/virtualenvs/niid-bench-orV17zke-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/namvq/.cache/pypoetry/virtualenvs/niid-bench-orV17zke-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
params = net.state_dict()

In [11]:
for key in params:
    print(params[key].type())

torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.FloatTensor
torch.FloatTensor
t

In [ ]:
def local_train(client, global_model, config, trainloader):
    """Huấn luyện mô hình trên một client cụ thể."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = copy.deepcopy(global_model).to(device) # Sử dụng bản sao mô hình toàn cục
    net.train()
    
    # Sử dụng SGD với learning rate = 1e-3 và momentum = 0.9
    optimizer = optim.SGD(
        # net.parameters(),
        filter(lambda p: p.requires_grad, net.parameters()), 
        lr=config.learning_rate, 
        momentum=config.momentum)
    tau = 0
    print(f"Training on client {client}, device: {device}")
    # Huấn luyện mô hình trên client
    for epoch in range(config.num_epochs):
        for batch_idx, (data, target) in enumerate(trainloader[client]):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = net(data)
            loss = torch.nn.CrossEntropyLoss()(output, target)
            loss.backward()
            optimizer.step()
            tau += 1  # Tăng số lần cập nhật

    a_i = (tau-config.momentum * (1-pow(config.momentum, tau)) / (1 - config.momentum)) / (1 - config.momentum)
    global_model_para = global_model.state_dict()
    net_para = net.state_dict()
    norm_grad = copy.deepcopy(global_model.state_dict())
    for key in norm_grad:
        norm_grad[key] = torch.true_divide(global_model_para[key] - net_para[key], a_i)
    

    return a_i, norm_grad